In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
from torch import nn

from PIL import Image
from pathlib import Path
import random
import matplotlib.pyplot as plt

import os

C:\Users\alexa\AppData\Local\Temp\ipykernel_27344\939756875.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

Original Challenge: https://challenge2018.isic-archive.com

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T
[1] Noel Codella, Veronica Rotemberg, Philipp Tschandl, M. Emre Celebi, Stephen Dusza, David Gutman, Brian Helba, Aadi Kalloo, Konstantinos Liopyris, Michael Marchetti, Harald Kittler, Allan Halpern: “Skin Lesion Analysis Toward Melanoma Detection 2018: A Challenge Hosted by the International Skin Imaging Collaboration (ISIC)”, 2018; https://arxiv.org/abs/1902.03368

[2] Tschandl, P., Rosendahl, C. & Kittler, H. The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions. Sci. Data 5, 180161 doi:10.1038/sdata.2018.161 (2018).

For good notebook: https://www.kaggle.com/code/sid321axn/step-wise-approach-cnn-model-77-0344-accuracy

In [ ]:
general_path = Path("/kaggle/input/skin-cancer-mnist-ham10000")
walk_through_dir(general_path)

In [ ]:
df = pd.read_csv(general_path / 'HAM10000_metadata.csv')
print(f"The shape of the metadata is {df.shape}")
df.head()

In [2]:
summary_dict = {
    'dx_distribution': df['dx'].value_counts(),
    'dx_type_distribution': df['dx_type'].value_counts(),
    'age_distribution': df['age'].describe(),
    'sex_distribution': df['sex'].value_counts(),
    'localization_distribution': df['localization'].value_counts()
}
summary_dict

IndexError: Cannot choose from an empty sequence

In [ ]:
# Let's make a dictionary for each image with its metadata, just for convenience:
image_metadata_dict = df.set_index('image_id').T.to_dict()

In [ ]:
# Now, we can always look up the corresponding information from an image_id:
image_id = 'ISIC_0027419'
metadata = image_metadata_dict.get(image_id)
metadata

From the competition, we know that "Cases include a representative collection of all important diagnostic categories in the realm of pigmented lesions: Actinic keratoses and intraepithelial carcinoma / Bowen's disease (akiec), basal cell carcinoma (bcc), benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, bkl), dermatofibroma (df), melanoma (mel), melanocytic nevi (nv) and vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, vasc)."

Let's plot some of the information so that it becomes a bit more clear.

# Let's first look at the different types of pigmented lesions and how many we have available to us in the dataset.
import matplotlib.pyplot as plt
import seaborn as sns

dx_data = summary_dict['dx_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Distribution')
plt.xlabel('type of pigmented lesion')
plt.ylabel('frequency')
plt.show()

In [ ]:
dx_data = summary_dict['dx_type_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Type Distribution')
plt.xlabel('How the lesion type got confirmed')
plt.ylabel('frequency')
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(df['age'])
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
dx_data = summary_dict['dx_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Distribution')
plt.xlabel('type of pigmented lesion')
plt.ylabel('frequency')
plt.show()

In [ ]:
image_path_part_1 = general_path / Path("HAM10000_images_part_1")
path_list_part_1 = list(image_path_part_1.glob('*.jpg'))
len(path_list_part_1)

image_path_part_2 = general_path / Path("HAM10000_images_part_2")
path_list_part_2 = list(image_path_part_2.glob('*.jpg'))
len(path_list_part_2)

all_images_path = path_list_part_1 + path_list_part_2 
len(all_images_path)

In [ ]:
random_image_path = random.choice(path_list_part_1)
img = Image.open(random_image_path)

print(f"Random image path: {random_image_path}")
print(f"Image height: {img.height}")
print(f"Image width: {img.width}")
img

In [ ]:
# We can also show it with matplotlib

# Turn the image into an array
img_as_array = np.asarray(img)

# Plot the image with matplotlib
plt.figure(figsize=(10, 7))
plt.imshow(img_as_array)
plt.title("Image shape: {img_as_array.shape} -> [height, width, color_channels]")
plt.axis(False)

In [ ]:
images_pixels_8_8_path = general_path / Path("hmnist_8_8.csv")
images_pixels_8_8_path